In [ ]:
!pip install transformers accelerate peft --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.7 MB/s eta 0:00:00


Loading the LoRA Model trained previously

In [ ]:
!mkdir -p peft-dialogue-summary-checkpoint

!wget https://dlai-generative-ai.s3.amazonaws.com/models/peft-dialogue-summary-checkpoint/adapter_config.json -P peft-dialogue-summary-checkpoint/
!wget https://dlai-generative-ai.s3.amazonaws.com/models/peft-dialogue-summary-checkpoint/adapter_model.bin -P peft-dialogue-summary-checkpoint/


--2025-04-17 17:18:50--  https://dlai-generative-ai.s3.amazonaws.com/models/peft-dialogue-summary-checkpoint/adapter_config.json
Resolving dlai-generative-ai.s3.amazonaws.com (dlai-generative-ai.s3.amazonaws.com)... 54.231.199.129, 16.15.217.116, 52.217.92.220, ...
Connecting to dlai-generative-ai.s3.amazonaws.com (dlai-generative-ai.s3.amazonaws.com)|54.231.199.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 334 [application/json]
Saving to: ‘peft-dialogue-summary-checkpoint/adapter_config.json’

adapter_config.json 100%[===================>]     334  --.-KB/s    in 0s      

2025-04-17 17:18:51 (21.3 MB/s) - ‘peft-dialogue-summary-checkpoint/adapter_config.json’ saved [334/334]

--2025-04-17 17:18:51--  https://dlai-generative-ai.s3.amazonaws.com/models/peft-dialogue-summary-checkpoint/adapter_model.bin
Resolving dlai-generative-ai.s3.amazonaws.com (dlai-generative-ai.s3.amazonaws.com)... 54.231.199.129, 16.15.217.116, 52.217.92.220, ...
Connecting to dl

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, T5Tokenizer, T5ForConditionalGeneration

from peft import PeftModel, PeftConfig
import torch


In [ ]:

peft_model_path = "./peft-dialogue-summary-checkpoint"
peft_model = PeftModel.from_pretrained(base_model, peft_model_path).to("cuda")
peft_model.eval()


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=32, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
            

In [ ]:


model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = AutoTokenizer.from_pretrained(chat_model_name)
model = AutoModelForCausalLM.from_pretrained(chat_model_name).to("cuda")

if chat_tokenizer.pad_token is None:
    chat_tokenizer.pad_token = chat_tokenizer.eos_token



In [ ]:

summarizer_tokenizer = T5Tokenizer.from_pretrained("t5-small")  # Pre-trained T5 tokenizer

summarizer_model = T5ForConditionalGeneration.from_pretrained("/content/peft-dialogue-summary-checkpoint")


In [ ]:
def summarize_conversation(conversation_history):
    conversation_text = " ".join(conversation_history)

    inputs = summarizer_tokenizer(conversation_text, return_tensors="pt", max_length=512, truncation=True)

    summary_ids = summarizer_model.generate(inputs['input_ids'], max_length=150, num_beams=4, early_stopping=True)

    summary = summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:

def chat():
    print("Bot: Hi! How can I help you today?")
    chat_history = []

    while True:
        user_input = input("User: ")

        if user_input.lower() == 'exit':
            print("Bot: Goodbye! Have a great day.")
            break

        chat_history.append(user_input)

        input_text = " ".join(chat_history)
        inputs = tokenizer(input_text, return_tensors="pt")

        outputs = model.generate(inputs['input_ids'], max_new_tokens=100)

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        print(f"Bot: {response}")

        chat_history.append(response)

        summary = summarize_conversation(chat_history)
        print("\n--- Summary of Conversation So Far ---")
        print(summary)
        print("--------------------------------------\n")


In [ ]:
chat()


Bot: Hi! How can I help you today?
User: Hi, I am Anudeep.
Bot:  Hi, nice to meet you. Do you have any pets? I have a dog.

--- Summary of Conversation So Far ---
Anudeep has a dog.
--------------------------------------

User: I have a cat.
Bot:  I don't, but I would like to get a dog one day. What kind of cat is it?

--- Summary of Conversation So Far ---
Anudeep has a cat. Anudeep has a dog. Anudeep has a cat.
--------------------------------------

User: It is a feral. What do you do for a living?
Bot:  It's a Siamese cat. I work in an office. How about you?

--- Summary of Conversation So Far ---
Anudeep has a Siamese cat. Anudeep works in an office.
--------------------------------------

User: exit
Bot: Goodbye! Have a great day.
